In [1]:
using DataFrames, CSV, Tables, Plots
using JuMP, Gurobi

In [2]:
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


Gurobi.Env(Ptr{Nothing} @0x0000000081636f80, false, 0)

In [3]:
network = (CSV.File("./soc-sign-bitcoinotc.csv") |> Tables.matrix)[:, 1:3];

In [4]:
# n = size(network)[1]
n = floor(Int, maximum([maximum(network[:, 1]), maximum(network[:, 2])]))
A = zeros(Int8, n, n)
G = zeros(Int8, n, n)

6005×6005 Matrix{Int8}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  

In [53]:
e = size(network)[1]
for i=1:e
    if network[i,3] < 0
        continue
    end
    s = floor(Int, network[i,1])
    d = floor(Int, network[i,2])
    A[s, d] = network[i,3]
    G[s, d] = min(max(A[s, d] + rand(-10:0), 0), 10)
end

In [54]:
graph = (CSV.File("./graph.csv") |> Tables.matrix);
graph_reach = (CSV.File("./graph_reach.csv") |> Tables.matrix);

In [55]:
synthetic = (CSV.File("./synthetic.csv") |> Tables.matrix)
threshold = synthetic[:,6];
budget = synthetic[:,7];

In [56]:
threshold_1 = 0.3*threshold;

In [57]:
T = 6
B = 10000

10000

In [58]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV));
set_optimizer_attributes(model, "TimeLimit" => 1800, "OutputFlag" => 1)
@variable(model, x[1:n,1:T] >= 0, Bin); 
@variable(model, y[1:n,1:T] >= 0, Bin);
@constraint(model,  sum(x[:,1].*budget) <= B)
for t=2:T
    @constraint(model, [i=1:n], sum(x[:,t-1].*A[i,:]) >= threshold[i]*x[i,t])
    @constraint(model, [i=1:n], sum(x[:,t-1].*A[i,:]) <= threshold[i]+y[i,t]*(1-threshold[i])+sum(A[i,:])*x[i,t])
    @constraint(model, [i=1:n], x[i,t-1] <= x[i,t])
    @constraint(model, [i=1:n], sum(y[:,t-1].*G[i,:]) >= threshold_1[i]*y[i,t])
    @constraint(model, [i=1:n], sum(y[:,t-1].*G[i,:]) <= threshold_1[i]+x[i,t]*(1-threshold_1[i])+sum(G[i,:])*y[i,t])
    @constraint(model, [i=1:n], y[i,t-1] <= y[i,t])
    @constraint(model, [i=1:n], y[i,t] + x[i,t] <= 1)
end
@objective(model, Max, sum(x[:,T]))
optimize!(model)

Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 210176 rows, 72060 columns and 698020 nonzeros
Model fingerprint: 0xc444d9c3
Variable types: 0 continuous, 72060 integer (72060 binary)
Coefficient statistics:
  Matrix range     [2e-02, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 201233 rows and 68403 columns
Presolve time: 4.23s
Presolved: 8943 rows, 3657 columns, 38375 nonzeros
Variable types: 0 continuous, 3657 integer (3657 binary)
Found heuristic solution: objective 5.0000000

Root relaxation: objective 4.434452e+01, 329 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe

In [59]:
sum(value.(x[:,T]))

30.0

In [60]:
sum(value.(y[:,T]))

0.0

In [36]:
q1 = value.(x)
q2 = value.(y)
time = zeros(Int8, n)
comp = zeros(Int8, n)
for i in 1:n
    for t in 1:T
        if q1[i, t] == 1
            if time[i] == 0
                time[i] = t
            end
        end
        if q2[i, t] == 1
            if time[i] == 0
                time[i] = t
                comp[i] = 1
            end
        end
    end
end
opt_example = DataFrame(Time=time, Comp=comp)
CSV.write("./opt_comp.csv", opt_example)

"./opt_comp.csv"